In this documents, we'll check if all the library has been correctly installed. 

In [2]:
# -- IMPORTS START --
import matplotlib
import matplotlib.pyplot as plt

import os
import re
import warnings
import datetime
import pathlib
import glob
import numpy as np
import pandas as pd
import joblib
from scipy import signal
from sklearn import metrics

from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn import tree


from scipy.signal import butter, filtfilt, find_peaks
# -- IMPORTS END --

# enable zooming into graphs
%matplotlib inline
plt.rcParams['figure.figsize'] = [9, 6] # width, height in inches

# Part 1: 

In [3]:
def plot_xyz(data):

    # Set axis
    axis = ['x', 'y', 'z']

    # Generate datetime index
    start = pd.Timestamp('2023-01-01')

    fig, axs = plt.subplots(ncols=3, nrows=1)

    for i in range(len(axis)):

        # Select random window
        start = 0
        end = 10 * 100
        window = data.iloc[start:end]

        ax = axs.flat[i]

        # Plot data
        ax.set_xticklabels([])        
        ax.plot(window.index, window[axis[i]], label=axis[i])

        # Plot peaks
        # peak_mask = window['peaks'] != 0
        # ax.plot(window.index[peak_mask], window['accel_mag'][peak_mask], 'ro', label='Peaks')

        ax.legend()
        ax.set_title(f"Axis-{axis[i]}")

    fig.tight_layout()
    fig.show()

In [4]:
# Helper function to visualize model
def viz_tree(dt_model,features_frames,cnames):
    # Fix feature names as list
    feature_names = features_frames.columns.tolist()

    fig, ax = plt.subplots(figsize=(9,4))
    tree.plot_tree(dt_model,
                   feature_names=feature_names,
                   fontsize=7,
                   class_names=cnames,
                   filled=True,
                   ax=ax)

    plt.title('Decision Tree')
    plt.savefig('dt.png')

In [5]:
def calc_magnitude(data):

    # Calculate magnitude
    data['accel_mag'] = np.sqrt(data['x']**2 + data['y']**2 + data['z']**2) # absolute accel magnitude
    data['accel_mag'] = data['accel_mag'] - data['accel_mag'].mean() # detrend: "remove gravity"

    return data

In [6]:
def remove_noise(data,sampling_rate):
    from scipy.signal import butter, filtfilt, find_peaks

    # Low pass filter
    cutoff = 5 # Hz
    order = 2
    b, a = butter(order, cutoff/(sampling_rate/2), btype='lowpass')
    data['filtered_accel_mag'] = filtfilt(b, a, data['accel_mag'])

    return data

In [7]:
def add_features(window):
    features = {}
    features['avg'] = window['filtered_accel_mag'].mean()
    features['max'] = window['filtered_accel_mag'].quantile(1)
    features['med'] = window['filtered_accel_mag'].quantile(0.5)
    features['min'] = window['filtered_accel_mag'].quantile(0)
    features['q25'] = window['filtered_accel_mag'].quantile(0.25)
    features['q75'] = window['filtered_accel_mag'].quantile(0.75)
    features['std'] = window['filtered_accel_mag'].std()
    df = pd.DataFrame()
    df = df._append(features,ignore_index=True)
    return df

In [8]:
def train_decision_tree(frames):
    # Extract feature columns
    X = frames[['avg', 'max', 'med', 'min', 'q25', 'q75', 'std']]

    # Extract target column
    y = frames['activity']

    # Split data
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

    # Create model
    dt_model = DecisionTreeClassifier(criterion='entropy',max_depth=5).fit(X_train, y_train)
    dt_pred = dt_model.predict(X_test)

    # Evaluate on test set
    acc = dt_model.score(X_test, y_test)
    dt_cm = confusion_matrix(y_test, dt_pred, labels=dt_model.classes_)
    print(classification_report(y_test, dt_pred))
    print("Accuracy on test set:", acc)

    return dt_model,dt_cm,acc

In [9]:
# Function to extract windows and features
def extract_features(data, window_sec, sample_rate, activity):
	window_str = f"{window_sec}s"
	data['time'] = pd.to_datetime(data['time'])
	data.set_index('time', inplace=True)
	resampled_data = data.resample(window_str)
	all_features = []
	for time, window in resampled_data:
		features = add_features(window)
		new_row = {
			'avg': features.iloc[0]['avg'],
			'max': features.iloc[0]['max'],
			'med': features.iloc[0]['med'],
			'min': features.iloc[0]['min'],
			'q25': features.iloc[0]['q25'],
			'q75': features.iloc[0]['q75'],
			'std': features.iloc[0]['std'],
			'activity': activity #?
		}
		all_features.append(new_row)
	return pd.DataFrame(all_features)

In [10]:
def all_data_to_combined_csv(root, output_filename = 'all_data.csv'):
    all_data = []
    sampling_rate = 100
    window_sec = 5

    
    activity_folders = os.listdir(root)
    print(activity_folders)
    for folder in activity_folders:
        activity_files = glob.glob(f"{root}/{folder}/*.csv")

        for file in activity_files:
            df = pd.read_csv(file, parse_dates=['time'])
            df = calc_magnitude(df)
            df = remove_noise(df, sampling_rate)
            df_features = extract_features(df, window_sec, sampling_rate, folder)
            
            for index, row in df_features.iterrows():
                new_row = {
                    'avg': row['avg'],
                    'max': row['max'],
                    'med': row['med'],
                    'min': row['min'],
                    'q25': row['q25'],
                    'q75': row['q75'],
                    'std': row['std'],
                    'activity': row['activity']
                }
                all_data.append(new_row)

    print(all_data)
    all_data = pd.DataFrame(all_data)
    all_data.to_csv(f"{root}/{output_filename}")  
    return all_data

# Part 2: Collect Training Data Into A Combined CSV